In [9]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *
from collections import OrderedDict


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)

        # error = torch.mean(torch.abs(captured['psum_recovered'] - captured['next_input']))

        # After output = model(input), inside train() or validate()
        # if 'psum_recovered' in captured and 'next_input' in captured:
        #     # Apply ReLU to recovered psum to match next layer's input
        #     psum_relu = F.relu(captured['psum_recovered'])
        #     next_input = captured['next_input']
        #     error = torch.mean(torch.abs(psum_relu - next_input)).item()
        
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)

            # if 'psum_recovered' in captured and 'next_input' in captured:
            #     # Apply ReLU to recovered psum to match next layer's input
            #     psum_relu = F.relu(captured['psum_recovered'])
            #     next_input = captured['next_input']
            #     error = torch.mean(torch.abs(psum_relu - next_input)).item()
            
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(testloader)
images, labels = next(dataiter) ## If you run this line, the next data batch is called subsequently.

imshow(torchvision.utils.make_grid(images))


In [10]:
# This cell won't be given, but students will complete the training

lr = 0.02
weight_decay = 5e-4
epochs = 170
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()

criterion = nn.CrossEntropyLoss(label_smoothing=0.1).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.747 (0.747)	Data 0.143 (0.143)	Loss 2.5209 (2.5209)	Prec 5.469% (5.469%)


KeyboardInterrupt: 

In [11]:

fdir = 'result/'+str(model_name)+'/model_best.pth.tar'

checkpoint = torch.load(fdir)
model.load_state_dict(checkpoint['state_dict'])


criterion = nn.CrossEntropyLoss().cuda()

model.eval()
model.cuda()

print('Accuracy for base case')
prec = validate(testloader, model, criterion)
print('First conv layer’s weights’ absolute sum = ', model.features[0].weight.abs().sum())

Accuracy for base case
Test: [0/79]	Time 0.133 (0.133)	Loss 0.2514 (0.2514)	Prec 94.531% (94.531%)
 * Prec 92.020% 
First conv layer’s weights’ absolute sum =  tensor(117.6888, device='cuda:0', grad_fn=<SumBackward0>)


In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [12]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

captured = {}
def save_input(module, input, output):
    captured['x'] = input[0].detach()  # input is a tuple
    
conv_layer = model.features[29]   # your squeezed conv layer
hook = conv_layer.register_forward_hook(save_input)


model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for i, (data, target) in enumerate(testloader):
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        if i == 0:
            x_to_layer29 = captured['x']
            print("Captured input shape:", x_to_layer29.shape)
        
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)
print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

Captured input shape: torch.Size([128, 8, 4, 4])

Test set: Accuracy: 9202/10000 (92%)



In [4]:
#send an input and grap the value by using prehook like HW3


In [13]:
import torch
import torch.nn.functional as F
import numpy as np

conv_layer = model.features[29]  # your squeezed quantized conv layer
w_bit = 4
x_bit = 4


# ---------- Stochastic rounding function ----------
def stochastic_round(tensor):
    floor = torch.floor(tensor)
    prob = tensor - floor
    return floor + torch.bernoulli(prob)

# ---------- Quantized weights ----------
weight_q = conv_layer.weight_q                      # stored during training
w_alpha = conv_layer.weight_quant.wgt_alpha.data    

# Symmetric quantization delta (corrected)
w_delta = w_alpha / (2**(w_bit - 1) - 1)

# Use stochastic rounding instead of torch.round
weight_int = stochastic_round(weight_q / w_delta).to(torch.int32)
print(weight_int.size())
W_int= torch.reshape(weight_int, (weight_int.size(0),weight_int.size(1),-1))
print(W_int.size())
print(W_int)
Kij = 0;
W = W_int[:,:,Kij].T.detach().clone()
print(W)
print(W.size())
bit_precision = 4
file = open('weight_kij0.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(W.size(1)):  
    for j in range(W.size(0)): 
       if (W[7-j,i] < 0):
        W[7-j,i] = W[7-j,i] + 16
       W_bin = '{0:04b}'.format(round(W[7-j,i].item()))
       for k in range(bit_precision):
           file.write(W_bin[k])        
       #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()





# ---------- Quantized input activations ----------
x = captured['x']                                    # input tensor to layer 29
x_alpha = conv_layer.act_alpha.data

# Symmetric activations quantization (0 to α → ReLU)
x_delta = x_alpha / (2**x_bit - 1)

# Quantize using same function used during training
act_quant_fn = conv_layer.act_alq
x_q = act_quant_fn(x, x_alpha)

x_int = stochastic_round(x_q / x_delta).to(torch.int32)


print(x_int.size())
x_int_b0 = x_int[0,:,:,:].clone().detach()


padding = 1
stride = 1
array_size = 64 # row and column number

nig = range(x_int_b0.size(1))  ## ni group [0,1,...31]
njg = range(x_int_b0.size(2))  ## nj group
 
icg = range(int(W_int.size(1)))  ## input channel [0,...63]
ocg = range(int(W_int.size(0)))  ## output channel

######## Padding before Convolution #######
X_pad = torch.zeros(len(icg), len(nig)+padding*2, len(njg)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
X_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = x_int_b0.cuda()
X_pad = torch.reshape(X_pad, (X_pad.size(0), -1))  ## mergin ni and nj index into nij
print(X_pad.size())
# a_pad.size() = [64, (32+2pad)*(32+2pad)]
print(X_pad)

kijg = range(W_int.size(2)) # [0, .. 8]
ki_dim = int(math.sqrt(W_int.size(2)))  ## Kernel's 1 dim size







bit_precision = 4
file = open('activation_padded_new.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X_pad.size(1)):  # time step
    for j in range(X_pad.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X_pad[7-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()


bit_precision = 4
file = open('activation_final.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

#for i in range(X.size(1)):  # time step
 #   for j in range(X.size(0)): # row #
  #      X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
   #     for k in range(bit_precision):
    #        file.write(X_bin[k])        
     #   #file.write(' ')  # for visibility with blank between words, you can use
    #file.write('\n')
#file.close()

torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 9])
tensor([[[-7, -1,  2, -2, -5, -4, -5, -4,  0],
         [ 7,  7,  7, -7, -7, -7,  7,  7,  7],
         [-7,  7, -7,  7,  7,  7, -7, -7, -7],
         [ 7, -7,  7, -7, -7, -7,  7, -7, -7],
         [-7,  7, -7,  7,  7,  7, -7,  7,  7],
         [-7, -7, -7,  7,  7,  7,  7,  7,  7],
         [-7,  7,  7,  7,  7,  7, -7, -7, -7],
         [ 7,  7, -7,  7,  7, -7, -7,  7, -7]],

        [[-7, -7, -7, -7, -7, -7, -7, -7, -7],
         [-7, -7, -4, -7, -7, -7, -7, -7, -7],
         [-7, -6, -1, -2, -7, -5, -6, -6, -5],
         [-7, -7, -7, -7, -7, -4, -6, -7, -6],
         [-7, -3, -7, -7, -7, -7, -5, -6, -7],
         [-6, -7, -6, -7, -7, -7, -7, -7, -7],
         [-7, -7, -7, -7, -7, -7, -7, -7, -7],
         [-4, -7, -4, -2, -7, -7, -5, -2,  1]],

        [[-7, -7, -7, -4, -6, -5, -7, -7, -7],
         [-7, -7, -3, -7, -7, -2, -7, -7, -7],
         [-7, -7, -7, -2, -7, -7, -7, -7, -5],
         [-7, -7, -3, -7, -7, -7, -7, -6, -7],
         

19

In [14]:

psum = torch.zeros(8,36,9).cuda()

for kij in range(9):
    for nij in range(36):
        psum[:, nij, kij] = torch.matmul(W_int[:, :, kij].float().cuda().T, X_pad[:, nij])




kij = 0
print(x_int[0,:,:,:])
print(weight_int[0,:,:,:])
psum_int = psum[:,:,0]
print(psum_int)

tensor([[[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]],

        [[ 8, 11, 13,  4],
         [13, 15, 15, 15],
         [15, 15, 15, 14],
         [10, 15, 15,  7]],

        [[15, 15, 11, 14],
         [14,  8,  5, 13],
         [ 3,  2,  2,  6],
         [ 7,  6,  6, 11]],

        [[ 8, 13, 15,  6],
         [15, 15, 15,  9],
         [15, 15, 15,  4],
         [13, 14, 10,  0]],

        [[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  2,  0]],

        [[ 1,  0,  0,  5],
         [ 2,  0,  0,  7],
         [ 0,  0,  7, 10],
         [ 1,  5, 10, 14]],

        [[ 0,  0,  0, 10],
         [ 8,  5,  2, 15],
         [15, 15, 13, 15],
         [15, 15, 15, 15]],

        [[12, 13, 12,  8],
         [15, 15, 15, 14],
         [ 8,  8, 13, 15],
         [11, 11, 15, 15]]], device='cuda:0', dtype=torch.int32)
tensor([[[-7, -1,  2],
         [-2, -5, -4],
         [-5, -4,  0]],

        [[ 7,

In [15]:
bit_precision = 16
file = open('psum_kij_8.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

kij = 8

psum_tile = psum[:,:,kij]
print(psum_tile.size(1))

for i in range(psum_tile.size(1)):  
    for j in range(psum_tile.size(0)): 
       if (psum_tile[7-j,i] < 0):
        psum_tile[7-j,i] = psum_tile[7-j,i] + 2**15
       P_bin = '{0:016b}'.format(round(psum_tile[7-j,i].item()))
       for k in range(bit_precision):
           file.write(P_bin[k])        
       #file.write(' ')  # for visibility with blank between words, you can use
 
    file.write('\n')
file.close() #close file 

36


In [22]:
# ---------- INT psum simulation (Hardware-like) ----------
output_int = F.conv2d(x_int.float(), weight_int.float(), bias=None,
                      stride=conv_layer.stride,
                      padding=conv_layer.padding)

print(x_int[0,:,:,:])
print(weight_int[0,:,:,:])
print(output_int.size())
print(output_int[0,:,:,:]);
o_int_b0 = output_int[0,:,:,:].clone().detach()
print(o_int_b0.size())
O = torch.reshape(o_int_b0, (o_int_b0.size(0),-1)).float().cuda().clone().detach()
print(O.size())
print(O)

bit_precision = 16
file = open('Output_new_relu.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(O.size(1)):  # time step
    for j in range(O.size(0)): # row # 
        
       if (O[7-j,i] < 0):
           O[7-j,i] = 0
           print(O[7-j,i])
       O_bin_k = '{0:016b}'.format(round(O[7-j,i].item()))
       o_new = O[7-j,i]
       
       for k in range(bit_precision):
           file.write(O_bin_k[k])    
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()
print(o_new)

tensor([[[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]],

        [[ 8, 11, 13,  4],
         [13, 15, 15, 15],
         [15, 15, 15, 14],
         [10, 15, 15,  7]],

        [[15, 15, 11, 14],
         [14,  8,  5, 13],
         [ 3,  2,  2,  6],
         [ 7,  6,  6, 11]],

        [[ 8, 13, 15,  6],
         [15, 15, 15,  9],
         [15, 15, 15,  4],
         [13, 14, 10,  0]],

        [[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  2,  0]],

        [[ 1,  0,  0,  5],
         [ 2,  0,  0,  7],
         [ 0,  0,  7, 10],
         [ 1,  5, 10, 14]],

        [[ 0,  0,  0, 10],
         [ 8,  5,  2, 15],
         [15, 15, 13, 15],
         [15, 15, 15, 15]],

        [[12, 13, 12,  8],
         [15, 15, 15, 14],
         [ 8,  8, 13, 15],
         [11, 11, 15, 15]]], device='cuda:0', dtype=torch.int32)
tensor([[[-7, -1,  2],
         [-2, -5, -4],
         [-5, -4,  0]],

        [[ 7,

In [ ]:
output_recovered = torch.relu(output_int * (x_delta * w_delta))
print(output_recovered)

output_fp32 = torch.relu(conv_layer(x))

difference = torch.abs(output_fp32 - output_recovered)
print('Mean difference:', difference.mean().item())
np.savetxt("weight_int_layer29.txt", weight_int.cpu().numpy().flatten(), fmt="%d", delimiter=",")
np.savetxt("output_recovered.txt", output_recovered.cpu().numpy().flatten(), fmt="%d", delimiter=",")


import torch

def int_to_unsigned_bits(x, bits=4):
    """Convert integer x to unsigned binary string of length `bits` (0 to 2^bits-1)."""
    max_val = (1 << bits) - 1  # 15 for 4 bits
    x_clipped = max(min(int(x), max_val), 0)  # clip to 0..15
    return format(x_clipped, '0{}b'.format(bits))

def write_activation_stream_row_major_unsigned(x_int, filename="x_int_layer29_hw_row_unsigned.txt",
                                               batch_index=0, bits=4):
    """
    Converts quantized tensor x_int to 4-bit unsigned binary stream, **row by row**.
    x_int: [B, 8, H, W]
    batch_index: which batch to export
    bits: bit precision
    """
    assert x_int.shape[1] == 8, "This helper expects 8 channels."
    X = x_int[batch_index].cpu().detach().clone()  # [C=8, H, W]
    C, H, W = X.shape

    with open(filename, 'w') as f:
        f.write('# Activation stream (row-major, unsigned): batch_index={}, 8 channels, {}-bit\n'.format(batch_index, bits))
        f.write('# Each line: one cycle, channels 7->0 concatenated\n')
        f.write('# Scanning order: for h in range(H): for w in range(W)\n')

        for h in range(H):        # row-first
            for w in range(W):    # column inside the row
                line = ''
                for ch in reversed(range(C)):  # ch7 -> ch0
                    val = X[ch, h, w].item()
                    line += int_to_unsigned_bits(val, bits)
                f.write(line + '\n')

    print(f"Written hardware stream (row-major, unsigned) to {filename}")
    return filename

# Usage
write_activation_stream_row_major_unsigned(x_int, filename="x_int_layer29_hw_row_unsigned.txt", batch_index=0, bits=4)

# ---------- Function to convert signed int to 2's complement bits ----------
def int_to_signed_bits(x, bits=4):
    """Convert integer x in range [-2^(bits-1), 2^(bits-1)-1] to 2's complement binary string."""
    min_val = -(1 << (bits-1))
    max_val = (1 << (bits-1)) - 1
    x_clipped = max(min(int(x), max_val), min_val)
    if x_clipped < 0:
        x_clipped = (1 << bits) + x_clipped
    return format(x_clipped, '0{}b'.format(bits))

# ---------- Write weights row-major for weight-stationary array ----------
def write_weights_row_major_signed(weight_int, filename="weight_layer29_hw_row.txt", bits=4):
    """
    Converts conv weights to 4-bit signed binary stream, row by row (input channel dimension is row).
    weight_int: [out_channels, in_channels, kH, kW]
    """
    O, I, kH, kW = weight_int.shape
    with open(filename, 'w') as f:
        f.write(f'# Weight stream (row-major, signed): {O}x{I}x{kH}x{kW}, {bits}-bit\n')
        f.write('# Each line: one cycle, input channels 0->7 concatenated\n')
        # For weight-stationary: scan output channels (rows) and kernel spatial positions
        for oc in range(O):
            for ic in range(I):
                for kh in range(kH):
                    for kw in range(kW):
                        val = weight_int[oc, ic, kh, kw].item()
                        bits_str = int_to_signed_bits(val, bits)
                        f.write(bits_str)
                    f.write('\n')  # each line is one kernel element across channels
    print(f"Written weight stream (row-major, signed) to {filename}")
    return filename

# ---------- Usage ----------
write_weights_row_major_signed(weight_int, filename="weight_layer29_hw_row.txt", bits=4)

# ---------- Write INT or Recovered outputs ----------
def write_output_stream_row_major(output_tensor, filename="output_hw_row.txt", bits=4, signed=False, batch_index=0):
    """
    Converts output tensor to 4-bit row-major binary stream for hardware.
    output_tensor: [B, C, H, W]
    signed: whether to use signed 2's complement (-8..7) or unsigned (0..15)
    """
    X = output_tensor[batch_index].cpu().detach().clone()  # [C, H, W]
    C, H, W = X.shape

    if signed:
        int_to_bits = int_to_signed_bits
    else:
        # unsigned helper
        def int_to_bits(x, bits=4):
            x_clipped = max(min(int(round(x)), (1<<bits)-1), 0)
            return format(x_clipped, '0{}b'.format(bits))

    with open(filename, 'w') as f:
        f.write(f'# Output stream (row-major, {"signed" if signed else "unsigned"}): {C}x{H}x{W}, {bits}-bit\n')
        f.write('# Each line: one cycle, channels 7->0 concatenated\n')
        for h in range(H):        # row-first
            for w in range(W):    # column inside row
                line = ''
                for ch in reversed(range(C)):  # ch7->ch0
                    val = X[ch, h, w].item()
                    line += int_to_bits(val, bits)
                f.write(line + '\n')

    print(f"Written output stream (row-major) to {filename}")
    return filename

# ---------- Usage ----------
write_output_stream_row_major(output_int, filename="output_int_hw_row.txt", bits=4, signed=True)
write_output_stream_row_major(output_recovered, filename="output_recovered_hw_row.txt", bits=4, signed=False)

tensor([[[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]],

        [[ 8, 11, 13,  4],
         [13, 15, 15, 15],
         [15, 15, 15, 14],
         [10, 15, 15,  7]],

        [[15, 15, 11, 14],
         [14,  8,  5, 13],
         [ 3,  2,  2,  6],
         [ 7,  6,  6, 11]],

        [[ 8, 13, 15,  6],
         [15, 15, 15,  9],
         [15, 15, 15,  4],
         [13, 14, 10,  0]],

        [[ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  2,  0]],

        [[ 1,  0,  0,  5],
         [ 2,  0,  0,  7],
         [ 0,  0,  7, 10],
         [ 1,  5, 10, 14]],

        [[ 0,  0,  0, 10],
         [ 8,  5,  2, 15],
         [15, 15, 13, 15],
         [15, 15, 15, 15]],

        [[12, 13, 12,  8],
         [15, 15, 15, 14],
         [ 8,  8, 13, 15],
         [11, 11, 15, 15]]], device='cuda:0', dtype=torch.int32)
tensor([[[-7, -1,  2],
         [-2, -5, -4],
         [-5, -4,  0]],

        [[ 7,

'output_recovered_hw_row.txt'

In [ ]:
import torch

# Make sure your model and captured hook are ready
x_q = captured['x']               # Already quantized activations from hook
weight_q = conv_layer.weight_q    # Already quantized weights

# Flatten and convert to CPU for saving
x_flat = x_q.detach().cpu().numpy().flatten()
w_flat = weight_q.detach().cpu().numpy().flatten()

# Save to text files
import os
save_dir = 'result/VGG16_quant/'
os.makedirs(save_dir, exist_ok=True)

x_path = os.path.join(save_dir, 'feature29_input_activations.txt')
w_path = os.path.join(save_dir, 'feature29_weights.txt')

# Save as float values
with open(x_path, 'w') as f:
    for val in x_flat:
        f.write(f"{val}\n")

with open(w_path, 'w') as f:
    for val in w_flat:
        f.write(f"{val}\n")

print(f"Saved {x_flat.shape[0]} input activations to {x_path}")
print(f"Saved {w_flat.shape[0]} weights to {w_path}")


In [ ]:
w_bit = 4
weight_q = conv_layer.weight_q # quantized value is stored during the training
w_alpha = conv_layer.weight_quant.wgt_alpha.data  # alpha is defined in your model already. bring it out here
w_delta = 2 * w_alpha / (2**w_bit - 1)    # delta can be calculated by using alpha and w_bit
weight_int = torch.round(weight_q / w_delta).to(torch.int32) # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

In [ ]:
x_bit = 4    
x = x = captured['x']   # input of the 2nd conv layer
x_alpha  = conv_layer.act_alpha.data
x_delta = 2 * x_alpha / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = torch.round(x_q / x_delta).to(torch.int32)
print(x_int) # you should see clean integer numbers 

In [ ]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  F.conv2d(x_int.float(), weight_int.float(), bias=None,
                      stride=conv_layer.stride, padding=conv_layer.padding)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int * (x_delta * w_delta)  # recover with x_delta and w_delta
print(output_recovered)

In [ ]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.features[3].weight_q 

output_ref = conv_ref(x)
print(output_ref)    

In [ ]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

In [ ]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.features[3].weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


In [ ]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model